# 과제 2: SageMaker Feature Store 사용

## Feature Store 옵션

Amazon SageMaker에서는 기본적으로 세 가지 방식을 통해 특성을 저장할 수 있습니다.
1. 전처리 단계가 완료되어 특성이 추가된 후 Amazon SageMaker Feature Store를 Amazon SageMaker Data Wrangler 대상으로 사용
2. 특성 정의, 특성 그룹 생성, SageMaker Feature Store로 데이터 수집을 실행하는 노트북을 SageMaker Data Wrangler에서 내보내기
3. 특성 정의, 특성 그룹 생성, SageMaker Feature Store로 데이터 수집을 실행하는 사용자 지정 노트북에서 SageMaker Python SDK 사용

다음 섹션에서 이러한 3가지 옵션에 대해 간략히 설명합니다.

### SageMaker Data Wrangler 대상으로 SageMaker Feature Store 사용

**Add destination** 옵션을 사용하여 Amazon SageMaker Studio에서 SageMaker Feature Store를 대상으로 추가할 수 있습니다. SageMaker Data Wrangler에서 전처리 단계를 완료한 후 흐름에서 **Add destination** 옵션을 선택할 수 있습니다. SageMaker Studio는 특성 그룹을 생성하는 방법을 안내하며, 전처리된 데이터를 SageMaker Feature Store로 수집하는 데 필요한 단계를 완료하는 방법도 안내합니다.

SageMaker Feature Store를 대상으로 추가하는 방법에 관한 자세한 내용은 [코드를 사용하지 않고 Amazon SageMaker에서 손쉽게 특성 생성 및 저장](https://aws.amazon.com/blogs/machine-learning/easily-create-and-store-features-in-amazon-sagemaker-without-code/#save_features_to_feature_store)을 참조하세요.

SageMaker Studio에서 특성 그룹을 생성하는 방법에 관한 자세한 내용은 [Amazon SageMaker Studio에서 Amazon SageMaker Feature Store 사용](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-use-with-studio.html)을 참조하세요.

### SageMaker Data Wrangler에서 노트북 내보내기

SageMaker Studio에서 **Export to** 옵션을 사용하여 특성 그룹을 생성할 수 있습니다. **Export to** 옵션을 사용할 때 특성 그룹을 생성하기 위해 실행해야 하는 모든 명령이 포함된 노트북을 생성할 수 있습니다.

몇 가지 요소만 사용자 지정하면 노트북을 실행하여 다음 작업을 수행할 수 있습니다.
- 데이터 집합을 기반으로 특성 정의 생성
- 특성 정의를 사용하여 특성 그룹 생성
- SageMaker Feature Store에 특성 그룹 저장
- 처리 작업의 입력 및 출력 설정

이 실습은 내보낸 노트북과 비슷하게 구성되어 있으며 내보낸 노트북의 앞부분에 해당하는 작업을 중점적으로 수행합니다. 특성 그룹에 데이터를 수집하고 온라인 저장소와 오프라인 저장소에서 레코드를 추출하는 방법을 살펴봅니다.

### 사용자 지정 노트북에서 SageMaker Python SDK 사용

이 실습에서는 특성 그룹을 생성하고 온라인 저장소와 오프라인 저장소에서 레코드를 추출합니다. 그리고 사용자 지정 노트북을 사용하여 SageMaker Feature Store의 작동 방식을 알아봅니다. 환경을 설정합니다. 그런 후에는 다음 과제를 완료합니다.

SageMaker Feature Store 설정
- 노트북 파일에서 특성 생성
- SageMaker Feature Store에서 특성 그룹 생성
- 특성 그룹이 생성되었는지 확인
- SageMaker Studio에서 특성 그룹 확인

온라인 및 오프라인 저장소 쿼리
- 특성 그룹에 데이터 수집
- 온라인 저장소에서 레코드 추출
- Amazon Athena를 사용하여 오프라인 저장소에서 레코드 추출

### 과제 2.1: 환경 설정

패키지 및 종속성을 설치합니다.

In [ ]:
#install-dependencies

import boto3
import json
import pandas as pd
import sagemaker
import sagemaker_datawrangler
import time
import uuid
import random
from sagemaker.session import Session
from sagemaker.feature_store.feature_definition import FeatureDefinition
from sagemaker.feature_store.feature_definition import FeatureTypeEnum
from sagemaker.feature_store.feature_group import FeatureGroup


region = boto3.Session().region_name
sess = boto3.Session(region_name=region)
bucket = sagemaker.Session().default_bucket()
role = sagemaker.get_execution_role()

처리된 고객 데이터 집합을 가져옵니다.

In [ ]:
#explore-dataset

column_list = ['income','age','education','education_num','capital_gain','capital_loss','hours_per_week','sex','workclass','marital_status','occupation','relationship','race']
lab_test_data = pd.read_csv('adult_data_processed.csv', names=(column_list), header=1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)

그런 다음, 테이블에서 데이터 집합 샘플을 검토합니다.

In [ ]:
#view-dataset

lab_test_data.dtypes
lab_test_data.head()

## SageMaker Feature Store 설정

모델을 훈련할 때 사용할 특성을 생성합니다. Amazon SageMaker Data Wrangler 실습에서 처리한 데이터를 가져와 SageMaker Feature Store를 사용하는 특성 그룹을 생성합니다.

- 노트북 파일에서 특성 생성
- SageMaker Feature Store에서 특성 그룹 생성
- 특성 그룹이 생성되었는지 확인
- SageMaker Studio에서 특성 그룹 확인

### 과제 2.2: 노트북 파일에서 특성 생성

특성 그룹을 생성하려면 **record_identifier_name** 및 **event_time_feature_name**용으로 할당할 열이 필요합니다. 이 요구 사항을 충족하려면 데이터 집합에 **record** 및 **event_time** 열을 추가합니다.
- **record_identifier_name**은 특성 그룹의 특성 정의에 정의되어 있는 특성의 이름 중 하나를 지칭합니다. 이 실습에서는 고유 ID 열 **record**를 만듭니다.
- **event_time_feature_name**은 특성 그룹에서 레코드를 생성하거나 업데이트하는 작업에 해당되는 새 이벤트가 발생하는 시점을 가리킵니다. 특성 그룹의 모든 레코드에는 해당하는 이벤트 시간이 있어야 합니다. 이 시간을 사용하면 시간 경과에 따른 레코드 변경 사항을 추적할 수 있습니다. 이 실습에서는 **event_time** 열을 생성합니다.

주요 용어, 레코드 식별자 이름 또는 이벤트 시간에 관한 자세한 내용은 [Amazon SageMaker Feature Store 개념](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-concepts.html)을 참조하세요.

In [ ]:
#add-required-columns

# Add record and event_time columns
current_time_sec = int(round(time.time()))
lab_test_data.insert(0, 'record', range(0, 0 + len(lab_test_data)))
lab_test_data.insert(1, 'event_time', [current_time_sec]*len(lab_test_data))
lab_test_data['record'] = lab_test_data['record'].astype('string')
lab_test_data['event_time'] = lab_test_data['event_time'].astype('float64')

# Set the record-and-event_time-feature-names
record_identifier_feature_name = 'record'
event_time_feature_name = 'event_time'

# View the dataset
print(lab_test_data.dtypes)
lab_test_data.head()

필요에 따라 데이터 처리 중에 SageMaker Python SDK 또는 SageMaker Studio를 사용하여 언제든지 새 특성을 추가할 수 있습니다.

이 실습에서는 데이터 전처리 단계의 일부로 SageMaker Python SDK를 사용하여 특성 하나를 추가합니다. 데이터 집합 내 열 2개의 가중치 기반 조합인 이 특성을 추가하면 모델 훈련 효율성을 높일 수 있습니다.

**age** 및 **hours_per_week** 열을 조합하여 workability 특성을 생성합니다. 이 특성을 추가하면 계속 일할 능력이 있는 고객을 파악할 수 있습니다.

In [ ]:
#add-feature

lab_test_data = lab_test_data.assign(
    workability = 0.5*lab_test_data.age + 0.5*lab_test_data.hours_per_week)

lab_test_data.head()

### 과제 2.3: SageMaker Feature Store에서 특성 그룹 생성

SageMaker Feature Store에 특성을 수집하려면 먼저 해당 특성 그룹에 속하는 모든 특성의 특성 정의(특성 이름 및 데이터 유형)를 정의합니다.

각 특성은 데이터 집합의 열에 해당합니다. 특성 그룹은 특성 컬렉션용으로 사전 정의된 스키마입니다. 특성 그룹의 각 특성에는 데이터 유형과 이름이 지정되어 있습니다. 특성 그룹의 각 레코드는 데이터 프레임의 행에 해당합니다. 특성 저장소는 특성 그룹 컬렉션입니다. 

SageMaker Feature Store에 관한 자세한 내용은 [Amazon SageMaker Feature Store를 사용하여 특성 생성, 저장 및 공유](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store.html)를 참조하세요.

특성 정의 만들기 프로세스를 시작하려면 각 특성의 스키마를 나열합니다.

In [ ]:
#list-column-schemas

column_schemas = [
    {
        "name": "record",
        "type": "string"
    },
    {
        "name": "event_time",
        "type": "float"
    },
    {
        "name": "income",
        "type": "string"
    },
    {
        "name": "age",
        "type": "long"
    },
    {
        "name": "education",
        "type": "float"
    },
    {
        "name": "education_num",
        "type": "float"
    },
    {
        "name": "capital_gain",
        "type": "long"
    },
    {
        "name": "capital_loss",
        "type": "long"
    },
    {
        "name": "hours_per_week",
        "type": "long"
    },
    {
        "name": "sex",
        "type": "float"
    },
    {
        "name": "workclass",
        "type": "array"
    },
    {
        "name": "marital_status",
        "type": "array"
    },
    {
        "name": "occupation",
        "type": "array"
    },
    {
        "name": "relationship",
        "type": "array"
    },
    {
        "name": "race",
        "type": "array"
    },
    {
        "name": "workability",
        "type": "float"
    }
]


스키마를 정의한 후에는 특성 정의의 입력을 생성합니다. float 및 long 데이터 집합 값(FRACTIONAL 및 INTEGRAL)의 유형 매핑을 설정합니다.

그런 다음, 정의한 스키마 내 모든 열의 **feature_name** 및 **feature_type** 값을 설정하여 특성 정의를 생성합니다.

In [ ]:
#create-feature-definitions

default_feature_type = FeatureTypeEnum.STRING
column_to_feature_type_mapping = {
    "float": FeatureTypeEnum.FRACTIONAL,
    "long": FeatureTypeEnum.INTEGRAL
}

feature_definitions = [
    FeatureDefinition(
        feature_name=column_schema['name'], 
        feature_type=column_to_feature_type_mapping.get(column_schema['type'], default_feature_type)
    ) for column_schema in column_schemas
]

SageMaker Feature Store에서 특성 그룹은 온라인 또는 오프라인 전용일 수도 있고 온라인과 오프라인에서 모두 사용할 수도 있습니다. 이 실습에서는 온라인 저장소와 오프라인 저장소를 둘 다 사용하므로 **enable_online_store**를 **True**로 설정합니다.
- 온라인 저장소는 기본적으로 읽기 지연 시간이 짧아야 하며(밀리초 단위) 쓰기 처리량이 높아야 하는 실시간 예측을 지원하는 데 사용됩니다.
- 오프라인 저장소는 기본적으로 배치 예측 및 모델 훈련에 사용됩니다. 오프라인 저장소는 과거 특성 데이터 저장과 액세스에 사용할 수 있는 추가 전용 저장소입니다.

특성 저장소가 온라인 및 오프라인 저장소로 동시에 설정되어 있으면 온라인 저장소에 수집하는 모든 특성이 오프라인 저장소에 복제됩니다. 

다음 설정에 대한 옵션을 지정하여 특성 그룹을 구성합니다.
- **feature_group_name**: 특성 그룹의 이름
- **feature_store_offline_s3_uri**: SageMaker Feature Store가 특성 그룹의 오프라인 저장소에서 데이터를 쓰는 Amazon Simple Storage Service(Amazon S3) 버킷 위치
- **enable_online_store**: 온라인 저장소 사용 여부 제어

In [ ]:
#configure-feature-store

# flow name and a unique ID for this export (used later as the processing job name for the export)
flow_name = "DataWranglerLab"
flow_export_id = f"{time.strftime('%d-%H-%M-%S', time.gmtime())}-{str(uuid.uuid4())[:8]}"
flow_export_name = f"flow-{flow_export_id}"
feature_group_name = f"FG-{flow_name}-{str(uuid.uuid4())[:8]}"
print(f"Feature Group Name: {feature_group_name}")

feature_store_offline_s3_uri = 's3://' + bucket
enable_online_store = True

특성 저장소를 구성했으므로 이제 AWS 리전을 설정하고 세션을 시작합니다. 그런 다음, SageMaker 클라이언트와 SageMaker Feature Store 런타임을 설정합니다. 마지막으로 SageMaker Feature Store 세션을 설정합니다.

특성 저장소 세션을 설정할 때는 **boto_session**, **sagemaker_client** 및 **sagemaker_featurestore_runtime_client** 값을 정의합니다.

In [ ]:
#set-up-sagemaker-feature-store-session

sagemaker_client = sess.client(service_name='sagemaker', region_name=region)
featurestore_runtime = sess.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=sess,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

앞에서 구성한 파라미터를 사용하여 특성 그룹을 초기화하고 Feature Store API를 호출하여 특성 그룹을 생성합니다.

In [ ]:
#initialize-feature-group

feature_group = FeatureGroup(
    name=feature_group_name, sagemaker_session=feature_store_session, feature_definitions=feature_definitions)

feature_group.create(
    s3_uri=feature_store_offline_s3_uri,
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=enable_online_store
)

### 과제 2.4: 특성 그룹이 생성되었는지 확인

이제 특성 그룹이 사용 가능한 상태여야 합니다. 특성 그룹이 정상적으로 생성되었는지 확인합니다.

Describe API를 사용하여 특성 그룹이 준비될 때까지 기다립니다. 이 함수는 Describe API가 반환하는 응답을 확인하고 응답의 상태가 **Created**가 될 때까지 기다립니다.

In [ ]:
#wait-for-describe

def wait_for_feature_group_creation_complete(feature_group):
    """Helper function to wait for the completions of creating a feature group"""
    response = feature_group.describe()
    status = response.get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        response = feature_group.describe()
        status = response.get("FeatureGroupStatus")

    if status != "Created":
        print(f"Failed to create feature group, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create feature group {feature_group.name}, status: {status}, reason: {failureReason}"
        )
    print(f"FeatureGroup {feature_group.name} successfully created.")

wait_for_feature_group_creation_complete(feature_group=feature_group)

ListFeatureGroups API를 사용하여 특성 그룹 목록을 표시합니다.

In [ ]:
#list-feature-groups

response = sagemaker_client.list_feature_groups()
print(json.dumps(response, indent=4, sort_keys=True, default=str))

### 과제 2.5: SageMaker Studio에서 특성 그룹 확인

SageMaker Python SDK를 사용하여 SageMaker 특성 그룹을 생성했습니다. 이제 SageMaker Studio의 특성 그룹을 검토하여 추가 세부 정보를 알아보십시오.

1. 노트북 왼쪽의 탐색 탭에서 **SageMaker Home** 아이콘을 선택합니다.

다음 단계에서는 SageMake Studio에서 새 탭이 열립니다. 여기서 설명하는 지침에 따라 작업을 진행하려면 다음 옵션 중 하나를 사용하세요.
- **옵션 1**: 탭을 나란히 표시합니다. 주 SageMaker Studio 창에서 분할 화면 보기를 생성하려면 **lab_5_ko_kr.ipynb** 탭을 옆쪽으로 끌거나 **lab_5_ko_kr.ipynb** 탭을 마우스 오른쪽 버튼으로 클릭하여 선택한 후 **New View for Notebook**을 선택합니다. 그러면 특성 그룹을 살펴볼 때 지침을 확인할 수 있습니다.
- **옵션 2**: SageMaker Studio 탭을 서로 전환하면서 지침에 따라 작업을 진행합니다. 특성 그룹 탐색을 완료한 후 **lab_5_ko_kr.ipynb** 탭을 선택하여 노트북으로 돌아옵니다.

2. **Data** 섹션을 확장하고 **Feature Store**를 선택합니다.
3. 방금 생성한 특성 그룹이 Feature Store 탭에 나타납니다. 특성 그룹에 관한 세부 정보를 검토할 수 있습니다. 추가 세부 정보를 확인하려면 이름이 **FG-DataWranglerLab-** 으로 시작되는 특성 그룹을 선택합니다. SageMaker Studio에서 SageMaker Feature Store를 탐색하면서 다음 세부 정보를 살펴봅니다.
    - **Feature**: 특성의 **Type** 그리고 특성이 생성된 시간(**event_time** 열 기준) 등의 정보를 비롯하여 특성 그룹의 모든 특성에 관한 설명이 제공됩니다.
    - **Details**: 특성 그룹의 메타데이터 개요가 제공됩니다. 여기서는 **Feature group status**, 이전 과제를 수행할 때 노트북에서 설정한 **Record identifier**, Online/Offline으로 설정되어 있는 **Store type**, Athena를 사용하여 오프라인 특성 저장소에서 데이터를 쿼리하는 데 사용할 수 있는 **Table name** 등의 메타데이터를 확인할 수 있습니다.
    - **Sample query**: 오프라인 특성 저장소에서 데이터를 쿼리하는 데 사용할 수 있는 다양한 샘플 쿼리가 제공됩니다.

### 과제 2.6: 온라인 및 오프라인 저장소 쿼리

특성 그룹을 생성했으므로 이제 특성 그룹에 데이터를 수집하고 온라인 저장소에서 레코드를 추출하며 Athena를 사용하여 오프라인 저장소에서 레코드를 추출합니다.

- 특성 그룹에 데이터 수집
- 온라인 저장소에서 레코드 추출
- Athena를 사용하여 오프라인 저장소에서 레코드 추출

### 과제 2.7: 특성 그룹에 데이터 수집

특성 그룹을 생성한 후에는 해당 그룹에 데이터를 추가합니다. 이렇게 하려면 SageMaker Python SDK에서 **ingest()** 를 사용하여 PutRecord API를 호출합니다. 특성 그룹을 처음으로 생성하거나 레코드를 더 추가하려는 경우에는 항상 특성 그룹에 레코드를 수집합니다. 

이 데이터 집합의 경우 데이터 수집은 3-5분 정도 걸립니다. 다음과 같은 출력이 표시되면 셀은 완료된 것입니다.

**IngestionManagerPandas(feature_group_name='FG-DataWranglerLab-13ee4f26', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7fdb7fccee60>, sagemaker_session=<sagemaker.session.Session object at 0x7fdb82a900d0>, max_workers=1, max_processes=1, profile_name=None, _async_result=None, _processing_pool=None, _failed_indices=[])**

In [ ]:
#ingest-records

feature_group.ingest(data_frame=lab_test_data, wait=True)

### 과제 2.8: 온라인 저장소에서 레코드 추출

온라인 저장소는 추론 과제에 특히 유용합니다. 특성 하위 집합을 빠르게 반환할 수 있기 때문입니다.

데이터를 수집했으므로 이제 **get_record**를 사용하여 온라인 저장소에서 레코드를 추출합니다.

In [ ]:
#get-record

record = random.randint(0, len(lab_test_data.index)-1)
sample_record = featurestore_runtime.get_record(FeatureGroupName=feature_group_name, RecordIdentifierValueAsString=str(record))

print(json.dumps(sample_record, indent=4, sort_keys=True, default=str))

이제 **batch_get_record**를 사용하여 특성 그룹에서 레코드를 몇 개 가져옵니다. 가져올 레코드는 이미 선택되어 있지만 **records_list**에 나와 있는 레코드를 마음대로 변경할 수 있습니다.

In [ ]:
#batch-get-record

records_list = ["7789", "5646", "309", "24528"]

batch_records = featurestore_runtime.batch_get_record(
    Identifiers=[
        {
            "FeatureGroupName": feature_group_name,
            "RecordIdentifiersValueAsString": records_list,
        }
    ]
)

print(json.dumps(batch_records, indent=4, sort_keys=True, default=str))

### 과제 2.9: Athena를 사용하여 오프라인 저장소에서 레코드 추출

온라인 저장소에서 레코드를 추출했으므로 이번에는 Athena를 사용하여 오프라인 저장소에서 레코드를 추출합니다. 

모델을 훈련하고 튜닝할 때 전체 데이터 집합을 쿼리할 수도 있고 추론용으로 레코드 하위 집합을 쿼리할 수도 있습니다. SageMaker Feature Store에는 모든 레코드의 이벤트 시간이 보존되므로 과거의 특정 시간에 해당하는 정확한 특성 집합을 사용하여 모델을 훈련할 수 있으며 해당 시간 이외의 데이터가 포함될 위험이 없습니다. 

오프라인 저장소에서 반환된 데이터 하위 집합을 변경하려면 쿼리를 어떻게 조정해야 할까요?

먼저 쿼리 설정을 선택합니다. 쿼리를 사용자 지정하여 특성 그룹에 저장된 데이터에서 원하는 하위 집합을 확인할 수 있습니다. 다음 쿼리는 기본적인 SELECT 쿼리 정의입니다.

In [ ]:
#query-settings
# Confirm the Athena settings are configured
try:
    boto3.client('athena').update_work_group(
        WorkGroup='primary',
        ConfigurationUpdates={
            'EnforceWorkGroupConfiguration':False
        }
    )
except Exception:
    pass

#Create the query
query = feature_group.athena_query()
table = query.table_name
query_string = f'SELECT * FROM "{table}" '
output_location = f's3://{bucket}/query_results/'

print(f'Athena query output location: \n{output_location}')

옵션을 설정한 후 쿼리를 실행하고 테이블 형식으로 결과를 표시합니다.

In [ ]:
#run-athena-query

query.run(query_string=query_string, output_location=output_location)
query.wait()
df = query.as_dataframe()
df.head()

### 마무리

축하합니다! SageMaker Feature Store를 사용하여 SageMaker Studio에서 SageMaker Python SDK로 특성 그룹의 특성 정의를 생성했습니다. 특성 그룹을 새로 생성했으므로 다음 실습에서 SageMaker Experiments를 사용하여 모델 훈련과 튜닝을 수행할 수 있습니다. 그리고 이 과정 뒷부분에서 추론 요청 처리를 위해 데이터를 추가할 때도 SageMaker Feature Store를 유용하게 활용할 수 있습니다. 온라인 저장소를 사용하므로 지연 시간이 짧은 GetRecord 기능이 제공되기 때문입니다. 다음 실습에서 이 고객 소득 데이터 집합을 계속 사용합니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫습니다.
- 실습 지침으로 돌아갑니다.